In [41]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')


In [2]:
!python -m spacy download en_core_web_md 

2022-06-06 10:23:40.531029: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-06 10:23:40.531130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
You should consider upgrading via the 'C:\Users\CHAITANYA AGRAWAL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 33.5/33.5 MB 3.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
data1= pd.read_csv("C:\\Users\CHAITANYA AGRAWAL\\OneDrive - IIT Delhi\\Desktop\\Projects\\Toronto\\Scraping the data\\Final script and data\\US\\US_2021.csv")
data2=pd.read_csv("C:\\Users\\CHAITANYA AGRAWAL\\OneDrive - IIT Delhi\\Desktop\\Projects\\Toronto\\Scraping the data\\Final script and data\\GB\\GB_2021.csv")
print(len(data1))
print(data1['text'][0])
print(data2['text'][0])
data1

16886
I know it's Superbowl week, but not enough attention is being paid to the fact that an entire professional sports team is out due to COVID protocols https://t.co/YfvehQTsmH
Every day is the same under  Covid 😞 until this weekend 🎉🥳 @SixNationsRugby @WelshRugbyUnion #SixNationsRugby @SuperBowl   @NFLUK #SuperBowl


,tweet_id,text,author_id,created_at,place_id,like_count,quote_count,reply_count,retweet_count,conversation_id,in_reply_to_user_id
0,1357131681799540739,"I know it's Superbowl week, but not enough att...",1193706258,2021-02-04T00:59:41.000Z,553011a282a18fb5,1,0,0,0,1357131681799540739,NaN
1,1357131627017682948,So excited @NKCSchools @ClardyElemNKC teacher ...,1599737341,2021-02-04T00:59:28.000Z,9a974dfc8efb32a0,7,0,0,1,1357131627017682948,NaN
2,1357131488521834501,@HedgehogBarnes @MichaelWight20 Have a good ev...,1168701223266115584,2021-02-04T00:58:55.000Z,27c45d804c777999,1,0,0,0,1357025463642841092,1.265781e+18
3,1357131474919710722,Super Bowl Sunday takeout wing Special! 5lbs ...,2858023997,2021-02-04T00:58:52.000Z,0122033a364ebdac,1,0,0,0,1357131474919710722,NaN
4,1357131429730254849,Still trying to fathom how Tom Brady is playin...,231557142,2021-02-04T00:58:41.000Z,092d7c3d0cba2fc2,5,0,1,1,1357131429730254849,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16881,1359276965916798976,Brady ain’t goat bc he didn’t win a super bowl...,395775438,2021-02-09T23:04:17.000Z,18810aa5b43e76c7,3,0,0,1,1359276965916798976,NaN
16882,1359276900758265856,"After Dallas trades away Dak, who gets a Super...",1076266429110984704,2021-02-09T23:04:01.000Z,7dda05213481260c,0,0,0,0,1359276900758265856,NaN
16883,1359276830902128641,@djhercules_787 and @joetorry killin' it at th...,3342210646,2021-02-09T23:03:45.000Z,dc62519fda13b4ec,0,0,0,0,1359276830902128641,NaN
16884,1359276425996599296,Losing in the divisional round &gt; losing in ...,1102915302474551297,2021-02-09T23:02:08.000Z,4015afc69a8a8506,11,0,2,0,1359276425996599296,NaN


In [4]:
data1['text'] = data1['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data2['text'] = data2['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [5]:
data1['text'][0]

"I know it's Superbowl week, but not enough attention is being paid to the fact that an entire professional sports team is out due to COVID protocols "

In [6]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [7]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to C:\Users\CHAITANYA
[nltk_data]     AGRAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
data1['text'] = data1['text'].apply(clean_text)
data1['Num_words_text'] = data1['text'].apply(lambda x:len(str(x).split())) 

print(len(data1))

max_sentence_length_1  = data1['Num_words_text'].max()
print(max_sentence_length_1)

data2['text'] = data2['text'].apply(clean_text)
data2['Num_words_text'] = data2['text'].apply(lambda x:len(str(x).split())) 

print(len(data2))

max_sentence_length_2  = data2['Num_words_text'].max()
print(max_sentence_length_2)


16886
80
2634
35


In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
data1['text']=data1['text'].apply(remove_stopwords)
data2['text']=data2['text'].apply(remove_stopwords)


In [10]:
data1['text'][0]

'know superbowl week enough attention paid fact entire professional sports team covid protocols'

In [11]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [12]:
print(data1['text'][1])
print(data2['text'][1])

excited nkcschools clardyelemnkc teacher chiefs cheerleader going superbowl students sunday thank alankmbc kmbc featuring ellie chiefskingdom runitback
always youtube version tallboyradio podcast drops audio version week features stars tandsshow talking football superbowl podcastlife podernfamily


In [13]:
text_list_1=data1['text'].tolist()
text_list_2=data2['text'].tolist()
print(text_list_1[1])
print(text_list_2[1])
tokenized_reviews_1 = lemmatization(text_list_1)
tokenized_reviews_2 = lemmatization(text_list_2)
print(tokenized_reviews_1[1])
print(tokenized_reviews_2[1])

excited nkcschools clardyelemnkc teacher chiefs cheerleader going superbowl students sunday thank alankmbc kmbc featuring ellie chiefskingdom runitback
always youtube version tallboyradio podcast drops audio version week features stars tandsshow talking football superbowl podcastlife podernfamily
['nkcschool', 'teacher', 'chief', 'cheerleader', 'superbowl', 'student']
['version', 'tallboyradio', 'podcast', 'audio', 'version', 'week', 'star', 'tandsshow', 'football', 'podcastlife']


In [36]:
dict={}
for arr in tokenized_reviews_1:
    for words in range(0,len(arr)):
        dict[arr[words]]=dict.get(arr[words],0)+1

In [38]:
word_freq = []
for key, value in dict.items():
    word_freq.append((value, key))

In [39]:
word_freq.sort(reverse=True) 
print(word_freq)

[(10418, 'super'), (10133, 'bowl'), (3890, 'superbowl'), (1276, 'year'), (981, 'game'), (971, 'good'), (885, 'chief'), (870, 'buccaneer'), (822, 'time'), (811, 'today'), (760, 'weekend'), (696, 'team'), (599, 'tomorrow'), (580, 'last'), (515, 'party'), (514, 'great'), (513, 'people'), (500, 'commercial'), (472, 'football'), (429, 'halftime'), (422, 'show'), (417, 'night'), (328, 'ready'), (309, 'happy'), (304, 'week'), (304, 'thing'), (302, 'first'), (301, 'tonight'), (296, 'sport'), (283, 'next'), (281, 'mahome'), (271, 'season'), (234, 'morning'), (224, 'wing'), (224, 'food'), (216, 'home'), (214, 'chiefskingdom'), (202, 'favorite'), (196, 'performance'), (192, 'many'), (191, 'free'), (191, 'fan'), (188, 'bucs'), (187, 'patrickmahome'), (186, 'gobucs'), (181, 'well'), (180, 'chance'), (179, 'champion'), (178, 'player'), (177, 'work'), (177, 'square'), (177, 'day'), (175, '️'), (174, 'coach'), (173, 'stadium'), (169, 'friend'), (167, 'score'), (166, 'order'), (161, 'ring'), (160, 'lif

In [14]:
dictionary_1 = corpora.Dictionary(tokenized_reviews_1)
dictionary_2 = corpora.Dictionary(tokenized_reviews_2)
doc_term_matrix_1 = [dictionary_1.doc2bow(rev) for rev in tokenized_reviews_1]
doc_term_matrix_2 = [dictionary_2.doc2bow(rev) for rev in tokenized_reviews_2]

In [23]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model_1 = LDA(corpus=doc_term_matrix_1, id2word=dictionary_1, num_topics=9, random_state=100,
                chunksize=1000, passes=50,iterations=100)
lda_model_2 = LDA(corpus=doc_term_matrix_2, id2word=dictionary_2, num_topics=9, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [24]:
lda_model_1.print_topics()


[(0,
  '0.229*"super" + 0.223*"bowl" + 0.024*"year" + 0.017*"last" + 0.016*"good" + 0.012*"team" + 0.012*"people" + 0.012*"night" + 0.010*"game" + 0.009*"weekend"'),
 (1,
  '0.087*"time" + 0.051*"show" + 0.051*"year" + 0.039*"next" + 0.037*"halftime" + 0.020*"coach" + 0.017*"half" + 0.017*"first" + 0.013*"love" + 0.012*"wing"'),
 (2,
  '0.030*"people" + 0.027*"home" + 0.026*"season" + 0.023*"️" + 0.018*"bad" + 0.015*"pandemic" + 0.015*"♂" + 0.014*"tombrady" + 0.010*"party" + 0.010*"twitter"'),
 (3,
  '0.080*"superbowl" + 0.027*"morning" + 0.021*"weekend" + 0.020*"tomorrow" + 0.016*"call" + 0.015*"good" + 0.013*"food" + 0.011*"order" + 0.011*"idea" + 0.009*"game"'),
 (4,
  '0.098*"super" + 0.098*"bowl" + 0.019*"game" + 0.014*"bucs" + 0.013*"week" + 0.012*"football" + 0.010*"field" + 0.010*"shit" + 0.010*"team" + 0.010*"chief"'),
 (5,
  '0.104*"super" + 0.102*"bowl" + 0.043*"commercial" + 0.018*"today" + 0.018*"youtube" + 0.015*"money" + 0.013*"score" + 0.012*"square" + 0.012*"chance" + 

In [25]:
lda_model_2.print_topics()

[(0,
  '0.070*"champion" + 0.022*"superbowl" + 0.020*"different" + 0.016*"watch" + 0.015*"nice" + 0.015*"franchise" + 0.011*"true" + 0.009*"lot" + 0.008*"party" + 0.008*"pregame"'),
 (1,
  '0.104*"superbowl" + 0.083*"buccaneer" + 0.060*"chief" + 0.046*"game" + 0.043*"mahome" + 0.031*"tonight" + 0.013*"gobucs" + 0.013*"time" + 0.012*"night" + 0.012*"🏻"'),
 (2,
  '0.062*"superbowl" + 0.047*"half" + 0.043*"time" + 0.035*"show" + 0.021*"weekend" + 0.018*"football" + 0.017*"today" + 0.015*"tomorrow" + 0.013*"sport" + 0.011*"congratulation"'),
 (3,
  '0.064*"superbowl" + 0.034*"sblv" + 0.027*"world" + 0.022*"tampabaybuccaneer" + 0.021*"people" + 0.020*"nfltwitter" + 0.019*"many" + 0.014*"sport" + 0.012*"commercial" + 0.011*"️"'),
 (4,
  '0.093*"superbowl" + 0.073*"team" + 0.018*"defence" + 0.016*"game" + 0.012*"bucs" + 0.011*"least" + 0.010*"amazing" + 0.009*"stadium" + 0.009*"pandemic" + 0.008*"happy"'),
 (5,
  '0.092*"year" + 0.062*"superbowl" + 0.032*"last" + 0.018*"ready" + 0.011*"right"

In [26]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis_1 = pyLDAvis.gensim_models.prepare(lda_model_1, doc_term_matrix_1, dictionary_1)
vis_1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.308076 -0.006262       1        1  30.901772
4     -0.232872 -0.001197       2        1  13.808370
7      0.110717 -0.292729       3        1  10.573229
5     -0.188325  0.023720       4        1   9.220303
1      0.077933  0.122415       5        1   8.721979
3      0.119965 -0.181226       6        1   7.719454
2      0.125457  0.132431       7        1   6.988286
8      0.135035  0.088899       8        1   6.343421
6      0.160167  0.113950       9        1   5.723187, topic_info=          Term         Freq        Total Category  logprob  loglift
7    superbowl  3438.000000  3438.000000  Default  30.0000  30.0000
20       super  9539.000000  9539.000000  Default  29.0000  29.0000
15        bowl  9329.000000  9329.000000  Default  28.0000  28.0000
38        time   772.000000   772.000000  Default  27.0000  27.0000
34   buccaneer   807.000000   807.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
72       thank    91.361756   117.569758   Topic9  -4.1553   2.6084
383      event    40.789136   139.887466   Topic9  -4.9617   1.6282
579    company    23.067527    41.394515   Topic9  -5.5317   2.2759
701       long    24.502831    73.668309   Topic9  -5.4714   1.7599
342        day    24.560735   125.325201   Topic9  -5.4690   1.2309

[403 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
748       5  0.987942         able
445       4  0.974735       actual
775       3  0.986457  advertising
63        1  0.623396      amazing
63        6  0.365726      amazing
...     ...       ...          ...
1728      3  0.958141            😍
286       3  0.978934            😭
5355      6  0.983047            🤘
1673      3  0.984109            🤣
4678      4  0.971403            🥴

[426 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 8, 6, 2, 4, 3, 9, 7])

In [27]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis_2 = pyLDAvis.gensim_models.prepare(lda_model_2, doc_term_matrix_2, dictionary_2)
vis_2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.384364 -0.046056       1        1  17.820478
1      0.079565 -0.227214       2        1  16.621919
8      0.080085 -0.112728       3        1  16.080507
2      0.022335  0.018827       4        1  15.243831
4      0.075876  0.035222       5        1   9.123639
3      0.036517  0.038439       6        1   8.225606
5      0.063340  0.021393       7        1   8.035940
7      0.016068  0.122716       8        1   4.482528
0      0.010578  0.149399       9        1   4.365552, topic_info=           Term        Freq       Total Category  logprob  loglift
18        super  600.000000  600.000000  Default  30.0000  30.0000
14         bowl  587.000000  587.000000  Default  29.0000  29.0000
93    buccaneer  176.000000  176.000000  Default  28.0000  28.0000
33    superbowl  990.000000  990.000000  Default  27.0000  27.0000
66         year  126.000000  126.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
258       party    4.681637   12.210975   Topic9  -4.7798   2.1727
2454          💍    2.440972    5.780411   Topic9  -5.4310   2.2693
33    superbowl   12.396361  990.907079   Topic9  -3.8060  -1.2498
920      second    3.125705   20.382280   Topic9  -5.1838   1.2564
540   breakfast    2.103134    6.269964   Topic9  -5.5800   2.0391

[453 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2056      6  0.883616         49er
249       4  0.673660     absolute
249       6  0.269464     absolute
2423      5  0.889536  achievement
1360      5  0.891445       actual
...     ...       ...          ...
2454      9  0.345996            💍
1189      2  0.821678            💪
1942      8  0.915056            🔥
1123      8  0.768301            😅
2820      1  0.973452            😮

[555 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 9, 3, 5, 4, 6, 8, 1])

In [ ]:

print('\nPerplexity: ', lda_model_1.log_perplexity(doc_term_matrix_1,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda_1 = CoherenceModel(model=lda_model_1, texts=tokenized_reviews_1, dictionary=dictionary_1 , coherence='c_v')
coherence_lda_1 = coherence_model_lda_1.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -8.195971462042008


In [20]:
from gensim.models.coherencemodel import CoherenceModel

In [21]:


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list_1, coherence_values_1 = compute_coherence_values(dictionary=dictionary_1, corpus=doc_term_matrix_1, texts=tokenized_reviews_1, start=2, limit=50, step=1)
model_list_2, coherence_values_2 = compute_coherence_values(dictionary=dictionary_2, corpus=doc_term_matrix_2, texts=tokenized_reviews_2, start=2, limit=50, step=1)

In [ ]:
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_1)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores


In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values_1):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary)
vis